# 제품 이상여부 판별 프로젝트


## 1. 데이터 불러오기


### 필수 라이브러리


In [1]:
import numpy as np
import pandas as pd

### 데이터 읽어오기


## 3. 모델링

공정별 데이터 구분

In [2]:
THRESHOLD = 0.3
RANDOM_STATE = 110

# csv 불러오기
train_data = pd.read_csv('./data/train_data_0817.csv')
test_data = pd.read_csv('./data/test_data_0817.csv')

데이터를 불러와 각 공정별로 (해당공정) + (공통변수) 를 하나의 데이터셋으로 만듬  
-> 각 공정별로 불량여부를 모델링하기 위함

In [3]:
# dam, fill1, fill2 공통 변수
var_dam_fill = [
    'Equipment_same_num',
    'PalletID_Collect_Result_encoded',
    'Production_Qty_Collect_Result',
    'WorkMode Collect Result'
]

In [4]:
# 전체 공통 변수
### correlation 확인을 위한 변수 리스트
var_all_corr = [
    'model_receip_encoded',
    'workorder_receip_encoded'
]

### train
var_all_train = [
    'target',
    'model_receip_encoded',
    'workorder_receip_encoded'
]

### test
var_all_test = [
    'Set ID',
    'target',
    'model_receip_encoded',
    'workorder_receip_encoded'
]

In [5]:
# '_Dam'을 포함하는 변수 선택
dam_variables = [var for var in train_data.columns if '_Dam' in var]

# train
final_columns_train = var_dam_fill + var_all_train + dam_variables
train_data_dam = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + dam_variables
test_data_dam = test_data[final_columns_test]

In [6]:
# '_Fill1'을 포함하는 변수 선택
fill1_variables = [var for var in train_data.columns if '_Fill1' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill1_variables
train_data_fill1 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill1_variables
test_data_fill1 = test_data[final_columns_test]

In [7]:
# '_Fill2'을 포함하는 변수 선택
fill2_variables = [var for var in train_data.columns if '_Fill2' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill2_variables
train_data_fill2 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill2_variables
test_data_fill2 = test_data[final_columns_test]

In [8]:
# '_AutoClave'을 포함하는 변수 선택
autoclave_variables = [var for var in train_data.columns if '_AutoClave' in var]

# train
final_columns_train = var_all_train + autoclave_variables
train_data_autoclave = train_data[final_columns_train]

# test 
final_columns_test = var_all_test + autoclave_variables
test_data_autoclave = test_data[final_columns_test]

In [9]:
# 각 DataFrame의 칼럼 수 계산
num_columns_train_data = train_data.shape[1]
num_columns_train_data_dam = train_data_dam.shape[1]
num_columns_train_data_autoclave = train_data_autoclave.shape[1]
num_columns_train_data_fill1 = train_data_fill1.shape[1]
num_columns_train_data_fill2 = train_data_fill2.shape[1]

num_columns_test_data = test_data.shape[1]
num_columns_test_data_dam = test_data_dam.shape[1]
num_columns_test_data_autoclave = test_data_autoclave.shape[1]
num_columns_test_data_fill1 = test_data_fill1.shape[1]
num_columns_test_data_fill2 = test_data_fill2.shape[1]

# 각 DataFrame의 칼럼 수 출력
print("----train data-----")
print(f"train_data DataFrame의 칼럼 수: {num_columns_train_data}")
print(f"train_data_dam DataFrame의 칼럼 수: {num_columns_train_data_dam}")
print(f"train_data_autoclave DataFrame의 칼럼 수: {num_columns_train_data_autoclave}")
print(f"train_data_fill1 DataFrame의 칼럼 수: {num_columns_train_data_fill1}")
print(f"train_data_fill2 DataFrame의 칼럼 수: {num_columns_train_data_fill2}")
print("----test data-----")
print(f"test_data DataFrame의 칼럼 수: {num_columns_test_data}")
print(f"test_data_dam DataFrame의 칼럼 수: {num_columns_test_data_dam}")
print(f"test_data_autoclave DataFrame의 칼럼 수: {num_columns_test_data_autoclave}")
print(f"test_data_fill1 DataFrame의 칼럼 수: {num_columns_test_data_fill1}")
print(f"test_data_fill2 DataFrame의 칼럼 수: {num_columns_test_data_fill2}")

----train data-----
train_data DataFrame의 칼럼 수: 38
train_data_dam DataFrame의 칼럼 수: 21
train_data_autoclave DataFrame의 칼럼 수: 8
train_data_fill1 DataFrame의 칼럼 수: 13
train_data_fill2 DataFrame의 칼럼 수: 13
----test data-----
test_data DataFrame의 칼럼 수: 39
test_data_dam DataFrame의 칼럼 수: 22
test_data_autoclave DataFrame의 칼럼 수: 9
test_data_fill1 DataFrame의 칼럼 수: 14
test_data_fill2 DataFrame의 칼럼 수: 14


모델 정의

In [10]:
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.ensemble import VotingClassifier

# 스레드홀드 설정
THRESHOLD = 0.3

# 모델 설정 및 하이퍼파라미터
models = {
    'et': ExtraTreesClassifier(),
    'rf': RandomForestClassifier(),
    'cat': CatBoostClassifier(),
    'lgbm': LGBMClassifier(),
    'xgb': XGBClassifier(),
    'dt': DecisionTreeClassifier(),
    'ada': AdaBoostClassifier()
}

def train_and_evaluate_model(model_name, data, **params):
    if model_name not in models:
        print(f"{model_name}은(는) 지원되지 않는 모델입니다.")
        return
    
    # 데이터셋 분할
    x_train, x_val, y_train, y_val = train_test_split(
        data.drop("target", axis=1),
        data["target"].map({'Normal': 0, 'AbNormal': 1}),
        test_size=0.2,
        shuffle=True,
        random_state=RANDOM_STATE,
    )

    # 모델 선택
    model = models[model_name].__class__()  # 새로운 모델 인스턴스 생성

    # 하이퍼파라미터 설정
    model.set_params(**params)

    # 모델 학습
    model.fit(x_train, y_train)

    # 데이터 이름을 자동으로 추출하기 위한 래퍼 함수
    data_name = [name for name in globals() if globals()[name] is data][0]

    # 예측
    y_val_pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
    y_val_pred = (y_val_pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측

    # 평가지표 계산
    f1 = f1_score(y_val, y_val_pred, average="binary")
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred, zero_division=0)
    recall = recall_score(y_val, y_val_pred)
    conf_matrix = confusion_matrix(y_val, y_val_pred)
    
    # 결과 출력
    print(f'{model_name} 모델이 {data_name} 데이터로 학습한 결과:')
    print(f'F1 Score: {f1}')
    print('---')
    print('Confusion Matrix:')
    print(conf_matrix)
    print('---')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print('\n')

    return model  # 학습된 모델 반환

def fit_all_train_data_function(model_name, data, **params):
    if model_name not in models:
        print(f"{model_name}은(는) 지원되지 않는 모델입니다.")
        return None  # 지원되지 않는 모델일 경우 None 반환
    
    # 모델 선택
    model = models[model_name].__class__()  # 새로운 모델 인스턴스 생성

    # 하이퍼파라미터 설정
    model.set_params(**params)

    # 모델 학습
    model.fit(data.drop("target", axis=1), data["target"].map({'Normal': 0, 'AbNormal': 1}))

    # 데이터 이름을 자동으로 추출하기 위한 래퍼 함수
    data_name = [name for name in globals() if globals()[name] is data][0]

    print(f'{model_name} 모델이 {data_name} 데이터로 학습 완료')
    return model  # 학습된 모델 반환

def voting_function(data, estimators, voting='hard', threshold=0.5):
    # 데이터셋 분할 # voting='hard'일 경우 threshold는 사용되지 않음
    x_train, x_val, y_train, y_val = train_test_split(
        data.drop("target", axis=1),
        data["target"].map({'Normal': 0, 'AbNormal': 1}),
        test_size=0.2,
        shuffle=True,
        random_state=RANDOM_STATE,
    )

    # VotingClassifier 설정
    voting_clf = VotingClassifier(estimators=estimators, voting=voting)

    # 모델 학습
    voting_clf.fit(x_train, y_train)

    if voting == 'soft':
        # 소프트 보팅의 경우 확률 예측
        y_val_pred_proba = voting_clf.predict_proba(x_val)[:, 1]
        y_val_pred = (y_val_pred_proba >= threshold).astype(int)
    else:
        # 하드 보팅의 경우 직접 예측
        y_val_pred = voting_clf.predict(x_val)

    # 평가지표 계산
    f1 = f1_score(y_val, y_val_pred, average="binary")
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred, zero_division=0)
    recall = recall_score(y_val, y_val_pred)
    conf_matrix = confusion_matrix(y_val, y_val_pred)
    
    # 결과 출력
    print(f'Voting Classifier로 학습한 결과:')
    print(f'F1 Score: {f1}')
    print('---')
    print('Confusion Matrix:')
    print(conf_matrix)
    print('---')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print('\n')

    return voting_clf  # 학습된 VotingClassifier 반환

def voting(preds_or_probs, method='soft', threshold=0.3):
    """
    하드 보팅 또는 소프트 보팅을 사용하여 최종 예측을 수행합니다.

    Parameters:
    preds_or_probs (list of np.array): 각 모델의 예측 배열 리스트 (하드 보팅) 또는 예측 확률 배열 리스트 (소프트 보팅)
    method (str): 'soft' 또는 'hard' 보팅 방법 선택
    threshold (float): 소프트 보팅 시 예측을 양성으로 간주할 확률 임계값

    Returns:
    np.array: 최종 예측 결과
    """
    if method == 'soft':
        # 소프트 보팅: 각 모델의 확률 평균 계산
        soft_voting_probs = np.mean(preds_or_probs, axis=0)
        # 최종 예측: 평균 확률에 대해 스레드 홀드 적용
        final_predictions = (soft_voting_probs >= threshold).astype(int)
    elif method == 'hard':
        # 하드 보팅: 각 모델의 예측을 모아서 다수결 원칙 적용
        preds = np.array(preds_or_probs)
        final_predictions = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=preds)
    else:
        raise ValueError("method 인자는 'soft' 또는 'hard'여야 합니다.")
    
    return final_predictions

공정별 모델 구축

lightgbm

In [108]:
model_Dam = fit_all_train_data_function(
    'lgbm', train_data_dam
    , n_estimators=1467
    , num_leaves=2545
    , max_depth=37
    , learning_rate=0.04353920224587149 
    , min_child_samples=83
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

model_AutoClave = fit_all_train_data_function(
    'lgbm', train_data_autoclave
    , n_estimators=1563
    , num_leaves=1885
    , max_depth=15
    , learning_rate=0.07033655355880039 
    , min_child_samples=158
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

model_Fill1 = fit_all_train_data_function(
    'lgbm', train_data_fill1
    , n_estimators=1452
    , num_leaves=1581
    , max_depth=22
    , learning_rate=0.002000452888170992 
    , min_child_samples=43
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

model_Fill2 = fit_all_train_data_function(
    'lgbm', train_data_fill2
    , n_estimators=1632
    , num_leaves=1426
    , max_depth=8
    , learning_rate=0.07487990991624197 
    , min_child_samples=90
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

model_All = fit_all_train_data_function(
    'lgbm', train_data
    , n_estimators=2383
    , num_leaves=2528
    , max_depth=343
    , learning_rate=0.04661896043153508
    , min_child_samples=209
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

# 예측에 필요한 데이터 분리
x_test_dam = test_data_dam.drop(["target", "Set ID"], axis=1)
x_test_autoclave = test_data_autoclave.drop(["target", "Set ID"], axis=1)
x_test_fill1 = test_data_fill1.drop(["target", "Set ID"], axis=1)
x_test_fill2 = test_data_fill2.drop(["target", "Set ID"], axis=1)
x_test_all = test_data.drop(["target", "Set ID"], axis=1)

# 예측 확률 리스트 (소프트 보팅용)
probs = [
    model_Dam.predict_proba(x_test_dam)[:, 1]
    , model_AutoClave.predict_proba(x_test_autoclave)[:, 1]
    , model_Fill1.predict_proba(x_test_fill1)[:, 1]
    , model_Fill2.predict_proba(x_test_fill2)[:, 1]
    , model_All.predict_proba(x_test_all)[:, 1]
]

# 소프트 보팅 결과
final_predictions = voting(probs, method='soft', threshold=0.24)
print(sum(final_predictions))

# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("./data/submission.csv")
df_sub["target"] = final_predictions

# df_sub['target'] 값을 문자열 레이블로 변환
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')

# 제출 파일 저장
df_sub.to_csv("./data/data0817_lgbm.csv", index=False)

lgbm 모델이 train_data_dam 데이터로 학습 완료
lgbm 모델이 train_data_autoclave 데이터로 학습 완료
lgbm 모델이 train_data_fill1 데이터로 학습 완료
lgbm 모델이 train_data_fill2 데이터로 학습 완료
lgbm 모델이 train_data 데이터로 학습 완료
498


xgb

In [109]:
model_Dam = fit_all_train_data_function(
    'xgb', train_data_dam
    , n_estimators = 1509
    , learning_rate = 0.06418917852996714
    , max_depth = 6
    , alpha = 0.00017309557032608048
    , gamma = 0.00398067155434722
    , reg_alpha = 0.756006595834120
    , reg_lambda = 0.3962538649486449
    , colsample_bytree =0.8752205595930229
    , subsample = 0.224637741333797
    , objective = 'binary:logistic'
    , tree_method = 'exact'
    , random_state=RANDOM_STATE
)

model_AutoClave = fit_all_train_data_function(
    'xgb', train_data_autoclave,
    n_estimators = 1539, 
    learning_rate = 0.026860419341696404, 
    max_depth = 14, 
    alpha = 1.9237525550524492e-05, 
    gamma = 2.2016346534611754e-05, 
    reg_alpha = 0.9148863773292526, 
    reg_lambda = 0.6194458787523232, 
    colsample_bytree = 0.902872150299903, 
    subsample = 0.10750014546599479,
    objective = 'binary:logistic', 
    tree_method = "exact", 
    random_state=RANDOM_STATE
)

model_Fill1 = fit_all_train_data_function(
    'xgb', train_data_fill1,
    n_estimators = 1707, 
    learning_rate = 0.0321470219836192, 
    max_depth = 7, 
    alpha = 7.368872823521818e-05, 
    gamma = 0.0007930035188326916, 
    reg_alpha = 0.644199314174124, 
    reg_lambda = 0.588270569327407, 
    colsample_bytree = 0.883929103208459, 
    subsample = 0.2534703342501092,
    objective = 'binary:logistic',
    tree_method = 'exact',
    random_state=RANDOM_STATE
)

model_Fill2 = fit_all_train_data_function(
    'xgb', train_data_fill2,
    n_estimators = 1998, 
    learning_rate = 0.030898693059763598, 
    max_depth = 8, 
    alpha = 0.0017554538174868774, 
    gamma = 0.0007257577447593802, 
    reg_alpha = 0.7581280398368035, 
    reg_lambda = 0.5872331353519633, 
    colsample_bytree = 0.56275606593282, 
    subsample = 0.8342870707789082,
    objective = 'binary:logistic',
    tree_method = 'exact',
    random_state=RANDOM_STATE
)

model_All = fit_all_train_data_function(
    'xgb', train_data,
    n_estimators = 2287, 
    learning_rate = 0.046904208411195795, 
    max_depth = 8, 
    alpha = 1.9343531171735368e-05, 
    gamma = 0.002118564280859176, 
    reg_alpha = 0.6827713868263061, 
    reg_lambda = 0.05035980721174918, 
    colsample_bytree = 0.8959193125044248, 
    subsample = 0.43471952905681815,
    objective = 'binary:logistic',  
    tree_method = "exact", 
    random_state=RANDOM_STATE
)

# 예측에 필요한 데이터 분리
x_test_dam = test_data_dam.drop(["target", "Set ID"], axis=1)
x_test_autoclave = test_data_autoclave.drop(["target", "Set ID"], axis=1)
x_test_fill1 = test_data_fill1.drop(["target", "Set ID"], axis=1)
x_test_fill2 = test_data_fill2.drop(["target", "Set ID"], axis=1)
x_test_all = test_data.drop(["target", "Set ID"], axis=1)


# 예측 확률 리스트 (소프트 보팅용)
probs = [
    model_Dam.predict_proba(x_test_dam)[:, 1]
    , model_AutoClave.predict_proba(x_test_autoclave)[:, 1]
    , model_Fill1.predict_proba(x_test_fill1)[:, 1]
    , model_Fill2.predict_proba(x_test_fill2)[:, 1]
    , model_All.predict_proba(x_test_all)[:, 1]
]

# 소프트 보팅 결과
final_predictions = voting(probs, method='soft', threshold=0.22)
print(sum(final_predictions))

# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("./data/submission.csv")
df_sub["target"] = final_predictions

# df_sub['target'] 값을 문자열 레이블로 변환
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')

# 제출 파일 저장
df_sub.to_csv("./data/data0817_xgb.csv", index=False)

xgb 모델이 train_data_dam 데이터로 학습 완료
xgb 모델이 train_data_autoclave 데이터로 학습 완료
xgb 모델이 train_data_fill1 데이터로 학습 완료
xgb 모델이 train_data_fill2 데이터로 학습 완료
xgb 모델이 train_data 데이터로 학습 완료
552


In [110]:
model_Dam = fit_all_train_data_function(
    'cat', train_data_dam,
    iterations=2083,
    learning_rate=0.023925705983940986,
    depth=11,
    l2_leaf_reg=0.05919257514332274,
    random_strength=7.259397831551647,
    bagging_temperature=5.39094676652102,
    border_count=234,
    scale_pos_weight=1.776413991309166,
    grow_policy='Lossguide',
    random_seed=RANDOM_STATE,
    eval_metric='F1',
    logging_level='Silent',
    boosting_type='Plain'
)

model_AutoClave = fit_all_train_data_function(
    'cat', train_data_autoclave,
    iterations = 2786, 
    learning_rate = 0.016342560305036093, 
    depth = 8, 
    l2_leaf_reg = 3.7187150890684246, 
    random_strength = 0.13164684607188099, 
    bagging_temperature = 9.823498597792092, 
    border_count = 158, 
    scale_pos_weight = 1.8735070170496537,
    grow_policy = 'SymmetricTree',
    random_state = RANDOM_STATE,
    eval_metric = 'F1',
    logging_level = 'Silent',
    boosting_type = 'Plain'
)

model_Fill1 = fit_all_train_data_function(
    'cat', train_data_fill1,
    iterations=1489,
    learning_rate=0.011481405951174946,
    depth=6,
    l2_leaf_reg=0.12082259365361882,
    random_strength=2.5111358694495056,
    bagging_temperature=2.06264856742851,
    border_count=331,
    scale_pos_weight=2.3505422278535173,
    grow_policy='Lossguide',
    random_seed=RANDOM_STATE,
    eval_metric='F1',
    logging_level='Silent',
    boosting_type='Plain'
)

model_Fill2 = fit_all_train_data_function(
    'cat', train_data_fill2,
    iterations=481,
    learning_rate=0.018742270357007457,
    depth=5,
    l2_leaf_reg=1.0871571324663387,
    random_strength=3.49632241801363,
    bagging_temperature=5.717049796462913,
    border_count=183,
    scale_pos_weight=3.4406776189795383,
    grow_policy='SymmetricTree',
    random_seed=RANDOM_STATE,
    eval_metric='F1',
    logging_level='Silent',
    boosting_type='Plain'
)

model_All = fit_all_train_data_function(
    'cat', train_data
    , iterations=1852
    , learning_rate=0.19270140909662298
    , depth=11
    , l2_leaf_reg=1.3258713856672897
    , random_strength=9.206702306227765
    , bagging_temperature=10.55412053150983
    , border_count=255
    , scale_pos_weight=1.5636124382284884
    , grow_policy = 'Lossguide'
    , random_seed = RANDOM_STATE
    , eval_metric = 'F1'
    , logging_level = 'Silent'
    , boosting_type = 'Plain'
)

# 예측에 필요한 데이터 분리
x_test_dam = test_data_dam.drop(["target", "Set ID"], axis=1)
x_test_autoclave = test_data_autoclave.drop(["target", "Set ID"], axis=1)
x_test_fill1 = test_data_fill1.drop(["target", "Set ID"], axis=1)
x_test_fill2 = test_data_fill2.drop(["target", "Set ID"], axis=1)
x_test_all = test_data.drop(["target", "Set ID"], axis=1)

# 예측 확률 리스트 (소프트 보팅용)
probs = [
    model_Dam.predict_proba(x_test_dam)[:, 1]
    , model_AutoClave.predict_proba(x_test_autoclave)[:, 1]
    , model_Fill1.predict_proba(x_test_fill1)[:, 1]
    , model_Fill2.predict_proba(x_test_fill2)[:, 1]
    , model_All.predict_proba(x_test_all)[:, 1]
]

# 소프트 보팅 결과
final_predictions = voting(probs, method='soft', threshold=0.3)
print(sum(final_predictions))

# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("./data/submission.csv")
df_sub["target"] = final_predictions

# df_sub['target'] 값을 문자열 레이블로 변환
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')

# 제출 파일 저장
df_sub.to_csv("./data/data0817_cat.csv", index=False)

cat 모델이 train_data_dam 데이터로 학습 완료
cat 모델이 train_data_autoclave 데이터로 학습 완료
cat 모델이 train_data_fill1 데이터로 학습 완료
cat 모델이 train_data_fill2 데이터로 학습 완료
cat 모델이 train_data 데이터로 학습 완료
391


et

In [111]:
model_Dam = fit_all_train_data_function(
    'et', train_data_dam
    , n_estimators = 2242
    , max_depth = 32
    , min_samples_split = 2
    , min_samples_leaf = 1
    , criterion = 'entropy'
    , bootstrap = False
    , random_state=RANDOM_STATE
)

model_AutoClave = fit_all_train_data_function(
    'et', train_data_autoclave,
    n_estimators = 2708,
    max_depth = 41,
    min_samples_split = 8,
    min_samples_leaf = 1,
    criterion = 'entropy',
    bootstrap = False,
    random_state=RANDOM_STATE
)

model_Fill1 = fit_all_train_data_function(
    'et', train_data_fill1,
    n_estimators = 1520,
    max_depth = 30,
    min_samples_split = 2,
    min_samples_leaf = 1,
    criterion = 'entropy',
    bootstrap = False,
    random_state=RANDOM_STATE
)

model_Fill2 = fit_all_train_data_function(
    'et', train_data_fill2
    , n_estimators = 1001
    , max_depth = 45
    , min_samples_split = 3
    , min_samples_leaf = 1
    , criterion = 'entropy'
    , bootstrap = False
    , random_state=RANDOM_STATE
)

model_All = fit_all_train_data_function(
    'et', train_data
    , n_estimators = 2884
    , max_depth = 56
    , min_samples_split = 3
    , min_samples_leaf = 1
    , criterion = 'entropy'
    , bootstrap = False
    , random_state=RANDOM_STATE
)


KeyboardInterrupt: 

In [ ]:
# 예측에 필요한 데이터 분리
x_test_dam = test_data_dam.drop(["target", "Set ID"], axis=1)
x_test_autoclave = test_data_autoclave.drop(["target", "Set ID"], axis=1)
x_test_fill1 = test_data_fill1.drop(["target", "Set ID"], axis=1)
x_test_fill2 = test_data_fill2.drop(["target", "Set ID"], axis=1)
x_test_all = test_data.drop(["target", "Set ID"], axis=1)

x_test_dam = x_test_dam.fillna(0)
x_test_autoclave = x_test_autoclave.fillna(0)
x_test_fill1 = x_test_fill1.fillna(0)
x_test_fill2 = x_test_fill2.fillna(0)
x_test_all = x_test_all.fillna(0)

# 예측 확률 리스트 (소프트 보팅용)
probs = [
    model_Dam.predict_proba(x_test_dam)[:, 1],
    model_AutoClave.predict_proba(x_test_autoclave)[:, 1],
    model_Fill1.predict_proba(x_test_fill1)[:, 1],
    model_Fill2.predict_proba(x_test_fill2)[:, 1],
    model_All.predict_proba(x_test_all)[:, 1]
]

# 소프트 보팅 결과
final_predictions = voting(probs, method='soft', threshold=0.22)
print(sum(final_predictions))

# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("./data/submission.csv")
df_sub["target"] = final_predictions

# df_sub['target'] 값을 문자열 레이블로 변환
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')

# 제출 파일 저장
df_sub.to_csv("./data/data0817_et.csv", index=False)

595


rf

In [ ]:
model_Dam = fit_all_train_data_function(
    'rf', train_data_dam
    , n_estimators = 1330
    , max_depth = 36
    , min_samples_split = 6
    , min_samples_leaf = 1
    , criterion = 'entropy'
    , bootstrap = False
    , random_state=RANDOM_STATE
)

model_AutoClave = fit_all_train_data_function(
    'rf', train_data_autoclave
    , n_estimators = 1103
    , max_depth = 36
    , min_samples_split = 8
    , min_samples_leaf = 1
    , criterion = 'entropy'
    , bootstrap = False
    , random_state=RANDOM_STATE
)

model_Fill1 = fit_all_train_data_function(
    'rf', train_data_fill1
    , n_estimators = 1861
    , max_depth = 91
    , min_samples_split = 7
    , min_samples_leaf = 5
    , criterion = 'entropy'
    , class_weight = 'balanced'
    , random_state=RANDOM_STATE
)

model_Fill2 = fit_all_train_data_function(
    'rf', train_data_fill2
    , n_estimators = 2663
    , max_depth = 100
    , min_samples_split = 6
    , min_samples_leaf = 1
    , criterion = 'entropy'
    , class_weight = 'balanced'
    , random_state=RANDOM_STATE
)

model_All = fit_all_train_data_function(
    'rf', train_data
    , n_estimators = 1082
    , max_depth = 54
    , min_samples_split = 6
    , min_samples_leaf = 1
    , criterion = 'entropy'
    , class_weight = 'balanced'
    , random_state=RANDOM_STATE
)

In [ ]:
import numpy as np
import pandas as pd

# 예측에 필요한 데이터 분리
x_test_dam = test_data_dam.drop(["target", "Set ID"], axis=1)
x_test_autoclave = test_data_autoclave.drop(["target", "Set ID"], axis=1)
x_test_fill1 = test_data_fill1.drop(["target", "Set ID"], axis=1)
x_test_fill2 = test_data_fill2.drop(["target", "Set ID"], axis=1)
x_test_all = test_data.drop(["target", "Set ID"], axis=1)

# NaN 값과 무한대 값 처리 함수
def handle_invalid_values(data):
    # NaN 값을 평균값으로 대체
    data = data.apply(lambda x: x.fillna(x.mean()), axis=0)
    # 무한대 값을 최대값으로 대체
    data = data.apply(lambda x: x.replace([np.inf, -np.inf], x.max()), axis=0)
    return data

# 각 데이터셋에 대해 NaN 값과 무한대 값 처리
x_test_dam = handle_invalid_values(pd.DataFrame(x_test_dam))
x_test_autoclave = handle_invalid_values(pd.DataFrame(x_test_autoclave))
x_test_fill1 = handle_invalid_values(pd.DataFrame(x_test_fill1))
x_test_fill2 = handle_invalid_values(pd.DataFrame(x_test_fill2))
x_test_all = handle_invalid_values(pd.DataFrame(x_test_all))


In [ ]:
# 예측 확률 리스트 (소프트 보팅용)
probs = [
    model_Dam.predict_proba(x_test_dam)[:, 1]
    , model_AutoClave.predict_proba(x_test_autoclave)[:, 1]
    , model_Fill1.predict_proba(x_test_fill1)[:, 1]
    , model_Fill2.predict_proba(x_test_fill2)[:, 1]
    , model_All.predict_proba(x_test_all)[:, 1]
]

# 소프트 보팅 결과
final_predictions = voting(probs, method='soft', threshold=0.28)
print(sum(final_predictions))

# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("./data/submission.csv")
df_sub["target"] = final_predictions

# df_sub['target'] 값을 문자열 레이블로 변환
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')

# 제출 파일 저장
df_sub.to_csv("./data/data0817_rf.csv", index=False)

864


ada

In [21]:
base_estimator = DecisionTreeClassifier(
    max_depth=22,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features=0.9449544624225188,
    random_state=42
)

model_Dam = fit_all_train_data_function(
    'ada', train_data_dam
    , base_estimator=base_estimator
    , n_estimators=439
    , learning_rate=0.30985993372769294
    , random_state=42
)

base_estimator = DecisionTreeClassifier(
    max_depth=13,
    min_samples_split=34,
    min_samples_leaf=5,
    max_features=0.7473309094470472,
    random_state=42
)

model_AutoClave = fit_all_train_data_function(
    'ada', train_data_autoclave
    , base_estimator=base_estimator
    , n_estimators=570
    , learning_rate=0.2040105705276999
    , random_state=42
)

base_estimator = DecisionTreeClassifier(
    max_depth=14,
    min_samples_split=33,
    min_samples_leaf=8,
    max_features=0.7113128413756866,
    random_state=42
)

model_Fill1 = fit_all_train_data_function(
    'ada', train_data_fill1
    , base_estimator=base_estimator
    , n_estimators=913
    , learning_rate=0.055237331816147595
    , random_state=42
)

base_estimator = DecisionTreeClassifier(
    max_depth=7,
    min_samples_split=13,
    min_samples_leaf=8,
    max_features=0.6266118401157937,
    random_state=42
)

model_Fill2 = fit_all_train_data_function(
    'ada', train_data_fill2
    , base_estimator=base_estimator
    , n_estimators=293
    , learning_rate=0.620377973483163
    , random_state=42
)

base_estimator = DecisionTreeClassifier(
    max_depth=6,
    min_samples_split=28,
    min_samples_leaf=7,
    max_features=0.7331591188366589,
    random_state=42
)

model_All = fit_all_train_data_function(
    'ada', train_data
    , base_estimator=base_estimator
    , n_estimators=677
    , learning_rate=0.6713565955468803
    , random_state=42
)


ada 모델이 train_data_dam 데이터로 학습 완료
ada 모델이 train_data_autoclave 데이터로 학습 완료
ada 모델이 train_data_fill1 데이터로 학습 완료
ada 모델이 train_data_fill2 데이터로 학습 완료
ada 모델이 train_data 데이터로 학습 완료


In [23]:

# NaN 값 확인 및 처리
x_test_dam = test_data_dam.drop(["target", "Set ID"], axis=1).fillna(0)
x_test_autoclave = test_data_autoclave.drop(["target", "Set ID"], axis=1).fillna(0)
x_test_fill1 = test_data_fill1.drop(["target", "Set ID"], axis=1).fillna(0)
x_test_fill2 = test_data_fill2.drop(["target", "Set ID"], axis=1).fillna(0)
x_test_all = test_data.drop(["target", "Set ID"], axis=1).fillna(0)
# 예측 확률 리스트 (소프트 보팅용)
probs = [
    model_Dam.predict_proba(x_test_dam)[:, 1]
    , model_AutoClave.predict_proba(x_test_autoclave)[:, 1]
    , model_Fill1.predict_proba(x_test_fill1)[:, 1]
    , model_Fill2.predict_proba(x_test_fill2)[:, 1]
    , model_All.predict_proba(x_test_all)[:, 1]
]

# 소프트 보팅 결과
final_predictions = voting(probs, method='soft', threshold=0.28)
print(sum(final_predictions))

# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("./data/submission.csv")
df_sub["target"] = final_predictions

# df_sub['target'] 값을 문자열 레이블로 변환
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')

# 제출 파일 저장
df_sub.to_csv("./data/data0817_ada.csv", index=False)

2629


### Hard voting

공정구분하여 학습한 개별 모델들에 대해서 hard voting

In [50]:

import pandas as pd
import numpy as np
from collections import Counter

def read_submission_files(file_paths):
    """
    제출 파일을 읽어와서 예측 결과를 반환합니다.

    Parameters:
    file_paths (list of str): 제출 파일 경로 리스트

    Returns:
    list of np.array: 각 제출 파일의 예측 결과 리스트
    """
    predictions = []
    for file_path in file_paths:
        df = pd.read_csv(file_path)
        preds = df['target'].apply(lambda x: 1 if x == 'AbNormal' else 0).values
        predictions.append(preds)
    return predictions

def hard_voting(preds):
    """
    하드 보팅을 사용하여 최종 예측을 수행합니다.

    Parameters:
    preds (list of np.array): 각 모델의 예측 배열 리스트

    Returns:
    np.array: 최종 예측 결과
    """
    preds = np.array(preds)
    
    # 각 샘플의 예측 결과를 문자열로 변환하여 리스트에 저장
    sample_predictions = [''.join(map(str, x)) for x in preds.T]
    
    # 각 예측 결과의 빈도수를 계산
    prediction_counts = Counter(sample_predictions)
    
    # 빈도수 출력
    for pred, count in prediction_counts.items():
        print(f"Prediction {pred}: {count} times")
    
    # 하드 보팅을 통해 최종 예측을 계산
    final_predictions = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=preds)
    return final_predictions

성능 좋은 3가지 모델(lgbm, xgb, cat)에 대해서 2번 넣어줌으로서 가중치를 주는 효과  
종합적으로 [2,2,2,1,1,1] 

In [112]:
import pandas as pd

# 공통 경로
common_path = "./data/"

# 제출 파일 이름 리스트
file_names = [
    "data0817_lgbm.csv"
    , "data0817_lgbm.csv"
    , "data0817_xgb.csv"
    , "data0817_xgb.csv"
    , "data0817_cat.csv"
    , "data0817_cat.csv"
    , "data0817_et.csv"
    , "data0817_rf.csv"
    , "data0817_ada.csv"   
    # 파일 추가 가능  <----- 파일 필요시 추가하세요!!
]

# 경로를 추가하는 함수
def add_common_path(file_names, common_path):
    return [common_path + file_name for file_name in file_names]

# 경로가 추가된 파일 리스트
file_paths = add_common_path(file_names, common_path)

# 제출 파일에서 예측 결과 읽어오기
predictions = read_submission_files(file_paths)

# 하드 보팅 결과
final_predictions_hard = hard_voting(predictions)

# 결과를 새로운 제출 파일로 저장할 파일 이름
output_file_name = "./data/ssdddddddddddddddddddd_submission.csv" # <----- 파일 이름을 변경하세요!!

# 결과를 새로운 제출 파일로 저장
df_sub = pd.read_csv(file_paths[0])
df_sub["target"] = final_predictions_hard
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')
df_sub.to_csv(output_file_name, index=False)

print(f"최종 제출 파일이 '{output_file_name}'로 저장되었습니다.")

Prediction 000000001: 1651 times
Prediction 000000000: 14590 times
Prediction 111111111: 240 times
Prediction 001100001: 41 times
Prediction 001100111: 21 times
Prediction 110000001: 20 times
Prediction 000000011: 173 times
Prediction 110011011: 5 times
Prediction 111100111: 51 times
Prediction 000000111: 116 times
Prediction 111100010: 10 times
Prediction 000000100: 22 times
Prediction 110000011: 15 times
Prediction 001100011: 31 times
Prediction 000011001: 21 times
Prediction 000000101: 68 times
Prediction 111100001: 19 times
Prediction 000000110: 12 times
Prediction 000011000: 11 times
Prediction 111100011: 36 times
Prediction 111111011: 38 times
Prediction 001111001: 3 times
Prediction 000011111: 13 times
Prediction 000000010: 29 times
Prediction 110011111: 10 times
Prediction 001111111: 13 times
Prediction 001100000: 18 times
Prediction 110000111: 13 times
Prediction 001100110: 4 times
Prediction 000011010: 2 times
Prediction 000011011: 11 times
Prediction 001100101: 3 times
Predi

In [115]:
df_sub['target'].value_counts()

Normal      16915
AbNormal      446
Name: target, dtype: int64

In [116]:
df_sub.head(10)

,Set ID,target
0,0001be084fbc4aaa9d921f39e595961b,Normal
1,0005bbd180064abd99e63f9ed3e1ac80,Normal
2,000948934c4140d883d670adcb609584,Normal
3,000a6bfd02874c6296dc7b2e9c5678a7,Normal
4,0018e78ce91343678716e2ea27a51c95,Normal
5,001fda4596f545d0a3b0ce85fbea77d2,Normal
6,0020734a7b29472298358ad58645a0c9,Normal
7,00234c5914cd4c4a888d13f8b3773135,Normal
8,00297b6c93e44d49ac534758a23dc74e,AbNormal
9,002d904240d84b188d410d16383a9c3a,Normal


우측 상단의 제출 버튼을 클릭해 결과를 확인하세요

=== Message ===  
작성하신 답안 제출이 완료되었습니다.  
Public Score : 0.232497

.